In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,sklearn,pandas
# %watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
# %watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
# %config IPCompleter.use_jedi = False

%matplotlib widget
# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal
import numpy as np
import copy
import functools

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try:
        x = x.strip()
        if x[0] == '/' or x[1] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                x = 'https://' + x
                x = requests.get(x)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                x = x.replace('github.com', 'raw.githubusercontent.com')
                mod = x.split('/')
                for s in ['/main/', '/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except Exception as err:
        # sys.stderr.write(f'request {x} : {err}')
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)


In [2]:
_IMPORT('gitee.com/qrsforever/nb_easy/easy_widget')

In [3]:
cos_prefix = 'https://datasets-1301930378.cos.ap-beijing.myqcloud.com/notebook/calibrate'

video_files = [
    f'{cos_prefix}/0049e1c813e9/20220908191302.mp4',
    f'{cos_prefix}/006229ae0bd1/20220908191004.mp4',
    f'{cos_prefix}/00da7bf0064b/20220908190758.mp4',
    f'{cos_prefix}/00dac8ecbfc2/20220908191451.mp4',
]

chessboard_images_path = './out/ipcam'

os.makedirs(chessboard_images_path, exist_ok=True)

In [4]:
def start_calibrate_camera_with_good_flags(ctx, calibrate, img_size, thresh_rate=1.25):
    flags = 0
    K = np.array([[max(img_size), 0, img_size[1]/2],[0, max(img_size), img_size[0]/2], [0, 0, 1]])
    D = np.zeros((5, 1))
    
    # An RMS error of 1.0 means that, on average, each of these projected points is 1.0 px away from its actual position. 
    # [0.0, 1.0] is good
    retval, mat, dist, rvecs, tvecs, \
    std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)

    ctx.logger(json.dumps({
        're-project-error': retval, 
        'mat': str(mat.round(3).tolist()),
        'dist': str(dist.round(3).tolist())}, indent=4))

    aspect_ratio = mat[0][0] / mat[1][1]
    if 1.0 - min(aspect_ratio, 1.0/aspect_ratio) < 0.01:
        flags += cv2.CALIB_FIX_ASPECT_RATIO
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'aspect_ratio': aspect_ratio,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))

    center_point_reldiff = max(abs(np.array(mat[0, 2], mat[1][2]) - np.array(img_size)/2) / np.array(img_size))
    if center_point_reldiff < 0.05:
        flags += cv2.CALIB_FIX_PRINCIPAL_POINT
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'center_point_reldiff': center_point_reldiff,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))

    error_threshold = thresh_rate * retval
    camera_matrix = mat
    dist_coeffs = dist
    error = retval

    ignore_flags = {
        'ignore_tangential_distortion': cv2.CALIB_ZERO_TANGENT_DIST,
        'ignore_k3': cv2.CALIB_FIX_K3,
        'ignore_k2': cv2.CALIB_FIX_K2,
        'ignore_k1': cv2.CALIB_FIX_K1
    }

    for k, v in ignore_flags.items():
        flags += v
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'ignore_type': k,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))
        if retval > error_threshold:
            continue
        camera_matrix = mat
        dist_coeffs = dist
        error = retval
    
    return error, camera_matrix, dist_coeffs

In [5]:
def _interact_update_samples(ctx, video_url, w_image_url):
    ctx.logger(f'_interact_update_samples({video_url})')
    mac = video_url.split('/')[-2]
    root_dir = f'{chessboard_images_path}/{mac}'
    options = []
    if os.path.isdir(root_dir):
        options = [(d.split('.')[0], f'{root_dir}/{d}') for d in os.listdir(root_dir) if d[-3:] == 'png']
    w_image_url.options = options

def _observe_video_snapshot(ctx, owner, old, new, w_video_url, w_sample_images, w_undist_image):
    ctx.logger('_observe_video_snapshot')
    if len(new) == 0:
        return
    mac = w_video_url.options[w_video_url.index][0].split('/')[0]
    ipath = f'{chessboard_images_path}/{mac}/{int(time.time())}.png'
    os.makedirs(os.path.dirname(ipath), exist_ok=True)
    with open(ipath, 'wb') as f:
        f.write(new)
    options = list(copy.copy(w_sample_images.options))
    options.append((os.path.basename(ipath), ipath))
    w_sample_images.options = options
    w_sample_images.index = len(options) - 1
    
    yaml_file = f'{os.path.dirname(ipath)}/intrinsics.yaml'
    if os.path.isfile(yaml_file):
        cv_file = cv2.FileStorage(yaml_file, cv2.FILE_STORAGE_READ)
        camera_matrix = cv_file.getNode('camera_matrix').mat()
        dist_coeffs = cv_file.getNode('dist_coeffs').mat()
        img_org = cv2.imdecode(np.frombuffer(new, dtype=np.uint8), cv2.IMREAD_COLOR)
        img_undist = cv2.undistort(img_org.copy(), camera_matrix, dist_coeffs)
        nbeasy_widget_display({'org': img_org, 'undist': img_undist}, w_undist_image, resize=True, fontscale=3)
    
def _onclick_remove_sample(ctx, owner, w_sample_images):
    ctx.logger('_onclick_remove_sample')
    index = w_sample_images.index
    options = list(copy.copy(w_sample_images.options))
    _, ipath = options.pop(index)
    if os.path.isfile(ipath):
        os.remove(ipath)
        
    w_sample_images.options = options
    length = len(options)
    if length == 0:
        return
    if index == length:
        index -= 1
    w_sample_images.index = index

def _onclick_check_corner(ctx, owner, w_image, chessboard, win_size, term_iters, term_eps):
    ctx.logger('_onclick_check_corner(%s, %s, %s, %s)' % (
        chessboard, win_size, term_iters, term_eps
    ))
    if not w_image.value or len(w_image.value) == 0:
        return
    squares_x, squares_y, square_size = chessboard
    pattern_size = (squares_x - 1, squares_y - 1)

    win_size, zero_zone = (win_size, win_size), (-1, -1)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, term_iters, term_eps)
    
    gray = cv2.cvtColor(w_image.image_data, cv2.COLOR_BGR2GRAY)
    found, corners = cv2.findChessboardCorners(gray, pattern_size)
    if found:
        better_corners = cv2.cornerSubPix(gray, corners, win_size, zero_zone, criteria)
        imgbgr = cv2.drawChessboardCorners(w_image.image_data, pattern_size, better_corners, found)
        nbeasy_widget_display(imgbgr, w_image, resize=True)
        
def _onclick_start_calibrate(ctx, owner, chessboard, win_size, term_iters, term_eps, w_sample_images, auto_finder):
    ctx.logger('_onclick_start_calibrate()')
    options = w_sample_images.options
    if len(options) == 0:
        return
    try:
        owner.disabled = True
        out_dir = os.path.dirname(options[0][1])

        squares_x, squares_y, square_size = chessboard
        pattern_size = (squares_x - 1, squares_y - 1)
        obj_points, img_points = [], []
        objp = np.zeros((np.prod(pattern_size), 3), np.float32)
        objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
        objp = objp * square_size

        win_size, zero_zone = (win_size, win_size), (-1, -1)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, term_iters, term_eps)

        ipaths = [o[1] for o in options]
        img_size = None
        for path in ipaths:
            name = os.path.basename(path)
            img = cv2.imread(path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            if img_size is None:
                img_size = gray.shape[::-1]
            found, corners = cv2.findChessboardCorners(gray, pattern_size)
            better_corners = cv2.cornerSubPix(gray, corners, win_size, zero_zone, criteria)
            obj_points.append(objp)
            img_points.append(better_corners)

        if auto_finder:
            error, camera_matrix, dist_coeffs = start_calibrate_camera_with_good_flags(
                ctx,
                functools.partial(cv2.calibrateCameraExtended, objectPoints=obj_points, imagePoints=img_points),
                img_size=img_size, thresh_rate=1.1) 
        else: 
            error, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
                imageSize=img_size, objectPoints=obj_points, imagePoints=img_points,
                cameraMatrix=None, distCoeffs=None,
                flags=cv2.CALIB_ZERO_TANGENT_DIST,
            )

        ctx.logger(f'start calibrate error: {error}')

        cv_file = cv2.FileStorage(f'{out_dir}/intrinsics.yaml', cv2.FILE_STORAGE_WRITE)
        cv_file.write('camera_matrix', camera_matrix)
        cv_file.write('dist_coeffs', dist_coeffs)
        cv_file.release()
    except Exception:
        ctx.logger(traceback.format_exc(limit=6))
    finally:
        owner.disabled = False

In [6]:
# fording

# _IMPORT('./easy_widget.py')

schema = {
    'type': 'page',
    'objs': [
        nbeasy_widget_hline(),
        {
            'type': 'V',
            'objs': [
                {
                    'type': 'V',
                    'name': 'ChessBoard',
                    'objs': [
                        nbeasy_widget_stringenum(
                            'cfg.chessboard_choice', 'Board Choise', default=2,
                            enums = [
                                ('A4-15mm-11x8', (11, 8, 15)),
                                ('A4-30mm-9x7', (9, 7, 30)),
                                ('A4-25mm-9x7', (9, 7, 25)),
                                ('A3-25mm-16x11', (16, 11, 25))],
                            ),
                    ],
                }, # end chessboard
                {
                    'type': 'H',
                    'objs': [
                        nbeasy_widget_int('cfg.subpix_win_size', 'Win Size', default=5),
                        nbeasy_widget_int('cfg.subpix_term_iters', 'Term Iters', default=500),
                        nbeasy_widget_float('cfg.subpix_term_eps', 'Term EPS', default=0.0001),
                    ]
                },
            ]
        },
        nbeasy_widget_hline(),
        {
            'type': 'H',
            'objs': [
                {
                    'type': 'V',
                    'objs': [
                        nbeasy_widget_video('__cfg.video', 'Video', video_files[0], width=640, format='url', height=352, btn_snapshot=True),
                        nbeasy_widget_stringenum(
                            'cfg.video_url', 'Video', 0,
                            enums=[('/'.join(url.split('/')[-2:]),  url) for url in video_files],
                            width=340, description_width=50
                        ),
                    ],
                    'align_items': 'center',
                    'width': '50%'
                },
                {
                    'type': 'V',
                    'objs': [
                        nbeasy_widget_button('__cfg.check_corner', 'Check', icon='check', width=100),
                        nbeasy_widget_image('__cfg.image', 'Image', '', width=640, height=352, e_onclick=False, plt=False),
                        {
                            'type': 'H',
                            'objs': [
                                nbeasy_widget_stringenum(
                                    'cfg.image_url', 'Image', 0,
                                    enums=[],
                                    width=300, description_width=0, 
                                    btn_next=True, btn_prev=True, btn_delete=False
                                ),
                                nbeasy_widget_button('__cfg.btn_del_sample', 'Remove', style='danger', icon='trash', width=100),
                            ],
                            'justify_content': 'center',
                        },
                    ],
                    'align_items': 'center',
                    'width': '50%'
                }
            ],
            'align_items': 'flex-end', # bottom align
        }, # video and image
        nbeasy_widget_hline(),
        {
            'type': 'V',
            'objs': [
                {
                    'type': 'H',
                    'objs': [
                        nbeasy_widget_button('cfg.start_calibrate', 'Calibrate', icon='start'),
                        nbeasy_widget_bool('cfg.auto_finder', 'Auto Find', False)
                    ],
                    'justify_content': 'center',
                },
                nbeasy_widget_image('__cfg.undist_image', 'Undist', width=640, height=352),
            ],
            'align_items': 'center',
        }
    ],
    'evts': [
        {
            'type': 'jsdlink',
            'objs': [
                {
                    'source': '__cfg.video:snapshot',
                    'target': '__cfg.image:value'
                },
                {
                    'source': 'cfg.video_url:value',
                    'target': '__cfg.video:url'
                },
                {
                    'source': 'cfg.image_url:value',
                    'target': '__cfg.image:url'
                }
            ],
        },
        {
            'type': 'interactiveX',
            'objs': [
                {
                    'handler': _interact_update_samples,
                    'params': {
                        'video_url': 'cfg.video_url',
                        'w_image_url': 'cfg.image_url',
                    }
                },
            ]
        },
        {
            'type': 'observe',
            'objs': [
                {
                    'handler': _observe_video_snapshot,
                    'params': {
                        'sources': ['__cfg.video:snapshot'],
                        'targets': [
                            'cfg.video_url', 
                            'cfg.image_url',
                            '__cfg.undist_image'
                        ]
                    }
                }
            ]
        },
        {
            'type': 'onclick',
            'objs': [
                {
                    'handler': _onclick_remove_sample,
                    'params': {
                        'sources': ['__cfg.btn_del_sample'],
                        'targets': ['cfg.image_url']
                    }
                },
                {
                    'handler': _onclick_check_corner,
                    'params': {
                        'sources': ['__cfg.check_corner'],
                        'targets': [
                            '__cfg.image',
                            'cfg.chessboard_choice:value',
                            'cfg.subpix_win_size:value',
                            'cfg.subpix_term_iters:value',
                            'cfg.subpix_term_eps:value',
                        ]
                    }
                },
                {
                    'handler': _onclick_start_calibrate,
                    'params': {
                        'sources': ['cfg.start_calibrate'],
                        'targets': [
                            'cfg.chessboard_choice:value',
                            'cfg.subpix_win_size:value',
                            'cfg.subpix_term_iters:value',
                            'cfg.subpix_term_eps:value',
                            'cfg.image_url',
                            'cfg.auto_finder:value'
                        ]
                    }
                }
            ]
        }
    ],
}

In [7]:
g_ctx = nbeasy_schema_parse(schema, border=False)

Box(children=(Box(children=(VBox(children=(HTML(value='<hr>'), VBox(children=(HTML(value="<b><font color='blac…

In [8]:
raise

RuntimeError: No active exception to reraise